In [25]:
import json
import requests
import praw
import numpy
import time
import pickle

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 10
wanted_count_sub = 10
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)
request_limit = 120
request_count = 0
request_sum = 0
sleep_time = 60
    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit, log):
    global request_count
    global request_sum

    if request_count >= request_limit:
        print("Sleeping...")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        request_count = 0
        print("Reseting request_count to 0")
        log.write("Reseting request_count to 0 \n")
        print("Resuming...")
        log.write("Resuming... \n")
        
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
    response_raw = requests.get(url)
    request_count += 1
    request_sum += 1
    

    print("Requests performed:\t\t\t", request_count)
    print("Total requests performed:\t\t", request_sum)
    print("Percent completed:\t\t\t", 555)
    
    log.write("Requests performed:\t\t\t {} \n".format(request_count))
    log.write("Total requests performed:\t\t {} \n".format(request_sum))
    log.write("Percent completed:\t\t\t {} \n".format(555))
    
    if response_raw.status_code != 200:
        print("Received {} error!".format(response_raw.status_code))
        log.write("Received {} error! \n".format(response_raw.status_code))
        print("Sleeping...")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        print("Reseting request_count...")
        log.write("Reseting request_count... \n")
        request_count = 0
        print("Performing request")
        log.write("Performing request \n")
        response_raw = requests.get(url)
        print(url, response_raw)
        log.write("{} {} \n".format(url, response_raw))
        response = response_raw.json()
        request_count += 1
        request_sum += 1

    print(url, response_raw)
    response = response_raw.json()

        
        
    
    #print(response)
    fetched_authors = getKeysFromDict(response['aggs']['author'])
    print("*******************")
    print()
    log.write("******************* \n")
    log.write("\n")

    time.sleep(1)
    return fetched_authors


def getMostActiveSubRedditForAuthor(author, log):
    global request_count
    global request_sum
    
    if request_count >= request_limit:
        print("Sleeping...")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        request_count = 0
        print("Reseting request_count to 0")
        print("Resuming...")
        
        log.write("Reseting request_count to 0 \n")
        log.write("Resuming... \n")
        
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
    response_raw = requests.get(url)
    
    request_count += 1
    request_sum += 1

    print("Requests performed:\t\t\t", request_count)
    print("Total requests performed:\t\t", request_sum)
    print("Percent completed:\t\t\t", 555)   
    
    log.write("Requests performed:\t\t\t {} \n".format(request_count))
    log.write("Total requests performed:\t\t {} \n".format(request_sum))
    log.write("Percent completed:\t\t\t {} \n".format(555)) 
    
    if response_raw.status_code != 200:
        print("Received {} error!".format(response_raw.status_code))
        log.write("Received {} error! \n".format(response_raw.status_code))
        print("Sleeping...")
        log.write("Sleeping... \n")
        time.sleep(sleep_time)
        print("Reseting request_count...")
        log.write("Reseting request_count... \n")
        request_count = 0
        print("Performing request")
        log.write("Performing request \n")
        response_raw = requests.get(url)
        print(url, response_raw)
        log.write("{} {} \n".format(url, response_raw))
        response = response_raw.json()
        request_count += 1
        request_sum += 1


    print(url, response_raw)
    response = response_raw.json()
    
    #print(response)
    fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
    print("Detected subreddits:\t\t\t", fetched_subreddits)
    log.write("Detected subreddits:\t\t\t \n".format(fetched_subreddits))
    for sub in used_subreddits:
            if (sub not in used_subreddits):
                used_subreddits.append(sub)
    print("*******************")
    print()
    log.write("******************* \n")
    log.write("\n")
    
    time.sleep(1)
    return fetched_subreddits

In [26]:
%%time

# Global variables
used_subreddits = []
used_authors = []
start = time.time()
log = open('log.txt', 'w+')


def get_relations(relational_array, subreddit, wanted_subcount):
    global request_count
    global request_sum
    global start
    time_elapsed = time.time() - start
    print("Time elapsed: \t\t\t\t", round(time_elapsed,2), "seconds")
    log.write("Time elapsed: \t\t\t\t {} seconds \n".format(round(time_elapsed,2)))

    
    if wanted_subcount == 0:
        return []
    
    # Add subreddit to used_subreddits 
    used_subreddits.append(subreddit)
    
    # Retrieve users from subreddit and add to request_count
    users = getMostActiveAuthorsInSubReddit(subreddit, log)

    # Retrieve subs from active users in subreddit
    for user in users:
            subs = getMostActiveSubRedditForAuthor(user, log)
            used_authors.append(user)

            # Enter new level
            for sub in subs:
                if sub != subreddit:
                    relational_array.append([subreddit, sub])

                if sub not in used_subreddits:
                    relational_array.extend(get_relations([], sub, wanted_subcount - 1))
    
    # Dump data to file
    pickle.dump(relational_array, open("relational_array.dat", "wb"))
    
    return relational_array


rels = get_relations([], 'The_Donald', 4)
rels
log.write("################################################################################\n")
log.write("################################################################################\n")
log.write("################################################################################\n")
log.write("SESSION ENDED\n")
log.write("################################################################################\n")
log.write("################################################################################\n")
log.write("################################################################################\n")
log.close()

Time elapsed: 				 0.0 seconds
Requests performed:			 1
Total requests performed:		 1
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=The_Donald&aggs=author&agg_size=12 <Response [200]>
*******************

Requests performed:			 2
Total requests performed:		 2
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&author=MAGA_MAGGING_MAGA&aggs=subreddit&agg_size=10 <Response [200]>
Detected subreddits:			 ['The_Donald', 'AmItheAsshole']
*******************

Time elapsed: 				 6.66 seconds
Requests performed:			 3
Total requests performed:		 3
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&subreddit=AmItheAsshole&aggs=author&agg_size=12 <Response [200]>
*******************

Requests performed:			 4
Total requests performed:		 4
Percent completed:			 555
https://api.pushshift.io/reddit/search/comment/?since=100d&author=tenaciousfall&aggs=subreddit&agg_size=10 <Response [200

KeyboardInterrupt: 

In [27]:
rel_array = pickle.load(open("relational_array.dat", "rb"))
rel_array

[['Random_Acts_Of_Amazon', 'Wishlist'],
 ['Random_Acts_Of_Amazon', 'AmazonFaeries'],
 ['Random_Acts_Of_Amazon', 'AskReddit'],
 ['Random_Acts_Of_Amazon', 'horror'],
 ['Random_Acts_Of_Amazon', 'bigtiddygothgf'],
 ['Random_Acts_Of_Amazon', 'comicbooks'],
 ['Random_Acts_Of_Amazon', 'Cyberpunk'],
 ['Random_Acts_Of_Amazon', 'xmen'],
 ['Random_Acts_Of_Amazon', 'GuessTheMovie'],
 ['Random_Acts_Of_Amazon', 'SquaredCircle'],
 ['Random_Acts_Of_Amazon', 'AmazonFaeries'],
 ['Random_Acts_Of_Amazon', 'pokemon'],
 ['Random_Acts_Of_Amazon', 'Keto_Food'],
 ['Random_Acts_Of_Amazon', 'RandomActsofCards'],
 ['Random_Acts_Of_Amazon', 'ketorecipes'],
 ['Random_Acts_Of_Amazon', 'nintendo'],
 ['Random_Acts_Of_Amazon', 'AskReddit'],
 ['Random_Acts_Of_Amazon', 'LifeOnPurple'],
 ['Random_Acts_Of_Amazon', 'RAoA_BookClub'],
 ['Random_Acts_Of_Amazon', 'millionairemakers'],
 ['Random_Acts_Of_Amazon', 'RandomActsOfGaming'],
 ['Random_Acts_Of_Amazon', 'RAoA_BookClub'],
 ['Random_Acts_Of_Amazon', 'Wishlist'],
 ['Random_

### 